## Объединение наборов данных: конкатенация и добавление в конец

In [1]:
import numpy as np
import pandas as pd

In [2]:
def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

In [3]:
make_df('ABCDE', range(5))

,A,B,C,D,E
0,A0,B0,C0,D0,E0
1,A1,B1,C1,D1,E1
2,A2,B2,C2,D2,E2
3,A3,B3,C3,D3,E3
4,A4,B4,C4,D4,E4


### Простая конкатенация с помощью метода pd.concat()

In [4]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

In [5]:
# pd.concat?
#     objs: 'Iterable[NDFrame] | Mapping[Hashable, NDFrame]',
#     axis=0,
#     join='outer',
#     ignore_index: 'bool' = False,
#     keys=None,
#     levels=None,
#     names=None,
#     verify_integrity: 'bool' = False,
#     sort: 'bool' = False,
#     copy: 'bool' = True,

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])

In [7]:
ser1

1    A
2    B
3    C
dtype: object

In [8]:
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [9]:
# concat DataFrame
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
df1

,A,B
1,A1,B1
2,A2,B2


In [10]:
df2

,A,B
3,A3,B3
4,A4,B4


In [11]:
pd.concat([df1, df2])
# по умолчанию конкатенация происходит построчно axis = 0

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [12]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
df3

,A,B
0,A0,B0
1,A1,B1


In [13]:
df4

,C,D
0,C0,D0
1,C1,D1


In [14]:
pd.concat([df3, df4])

,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
0,NaN,NaN,C0,D0
1,NaN,NaN,C1,D1


In [15]:
pd.concat([df3, df4], axis=1)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


#### Дублирование индексов
    Конкатенация Pandas сохраняет индексы, даже если они оказываются дублированы

In [16]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])

In [17]:
y.index = x.index

In [18]:
x

,A,B
0,A0,B0
1,A1,B1


In [19]:
y

,A,B
0,A2,B2
1,A3,B3


In [20]:
pd.concat([x, y])

,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


Может быть несколько способов решения этой проблемы:
###### Перехват повторов как ошибки:

In [ ]:
# если нужно гарантировать, чтобы индексы не перекрывались, то флаг verify_integrity

# pd.concat?
#     objs: 'Iterable[NDFrame] | Mapping[Hashable, NDFrame]',
#     axis=0,
#     join='outer',
#     ignore_index: 'bool' = False,
#     keys=None,
#     levels=None,
#     names=None,
#     verify_integrity: 'bool' = False,
#     sort: 'bool' = False,
#     copy: 'bool' = True,

In [22]:
# verify_integrity= true  то при перекрытии индексов возникнет ошибка
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print(e)

Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


###### Игнорирование индекса

In [ ]:
# ignore_index=True
# создает новый целочисленный индекс для повторов

In [23]:
pd.concat([x, y], ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


###### Добавление ключей мультииндекса
    - по сути преобразование в мультииндекс

In [24]:
pd.concat([x, y], keys=['x', 'y'])

A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

#### Конкатенация с использованием соединений:
    - На приктике у данных из разных источников могут быть разные имена столбцов

In [25]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
df5

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


In [26]:
df6

,B,C,D
3,B3,C3,D3
4,B4,C4,D4


In [29]:
pd.concat([df5, df6])
# поведение по умолчанию join='outer'

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


In [30]:
pd.concat([df5, df6], join='outer')

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


In [46]:
pd.concat([df5, df6], axis=0)

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


In [31]:
pd.concat([df5, df6], join='inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


In [33]:
# имена итоговых столбцов можно указать явно 
pd.concat([df5, df6], join_axes=[df5.columns])

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [66]:
# join_axes - параметр устарел
pd.concat([df5, df6]).loc[:,df5.columns]

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


#### Метод append()

In [67]:
df1

,A,B
1,A1,B1
2,A2,B2


In [68]:
df2

,A,B
3,A3,B3
4,A4,B4


In [69]:
df1.append(df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [ ]:
# метод append() библиотеки Pandas не изменяет исходный объект, а создает новый => жрёт память